In [2]:
import torch

# load the scripted model onto CPU if CUDA is not available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.jit.load("weights/flood_lstm_scripted.pt", map_location=device)
model.to(device)
model.eval()

# python3 src.ds.test.py

RecursiveScriptModule(
  original_name=FloodLSTMBinary
  (lstm): RecursiveScriptModule(original_name=LSTM)
  (fc): RecursiveScriptModule(original_name=Linear)
)

In [3]:
model.eval()


RecursiveScriptModule(
  original_name=FloodLSTMBinary
  (lstm): RecursiveScriptModule(original_name=LSTM)
  (fc): RecursiveScriptModule(original_name=Linear)
)

In [12]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.24.1%2Bcpu-cp310-cp310-manylinux_2_28_x86_64.whl (1.9 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.9.1%2Bcpu-cp310-cp310-manylinux_2_28_x86_64.whl (493 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn as nn

class FloodLSTMBinary(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int = 64, num_layers: int = 1, dropout: float = 0.0):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)         # (batch, seq_len, hidden_dim)
        last_out = out[:, -1, :]      # (batch, hidden_dim)
        logit = self.fc(last_out)     # (batch, 1)
        return logit.squeeze(-1)      # (batch,)


TIME_FEATURES = [
    "number_of_report_flood",  
    "water",                   
    # "total_report",          
]

STATIC_FEATURES = [
    "latitude",
    "longitude",
]

SEQ_LEN = 30

In [ ]:
import torch
import pickle
import pandas as pd
import numpy as np
# from flood_model_def import FloodLSTMBinary, TIME_FEATURES, STATIC_FEATURES, SEQ_LEN

# 1) Load scaler
with open("weights/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# 2) Build model and load CPU weights
input_dim = len(TIME_FEATURES) + len(STATIC_FEATURES)
model = FloodLSTMBinary(input_dim=input_dim, hidden_dim=64, num_layers=1)
state = torch.load("weights/model.pth", map_location="cpu")
model.load_state_dict(state)
model.eval()

/home/sirav/spark_env/lib/python3.10/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


FloodLSTMBinary(
  (lstm): LSTM(4, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [21]:
df = pd.read_csv("../../data/processed/flood_data_test.csv")
df["date"] = pd.to_datetime(df["date"])

date = df["date"].iloc[0]
subdistrict_name = df["subdistrict"].iloc[0]
sub = df[df["subdistrict"] == subdistrict_name].sort_values("date")

# build last SEQ_LEN days
feat_cols = TIME_FEATURES + STATIC_FEATURES
seq = sub[feat_cols].values[-SEQ_LEN:]

if len(seq) < SEQ_LEN:
    pad = np.zeros((SEQ_LEN - len(seq), seq.shape[1]))
    seq = np.vstack([pad, seq])

df_input = pd.DataFrame(seq[:, :len(TIME_FEATURES)], columns=TIME_FEATURES)
seq[:, :len(TIME_FEATURES)] = scaler.transform(df_input)

x = torch.tensor(seq, dtype=torch.float32).unsqueeze(0)  # (1, SEQ_LEN, input_dim)

try:
    model.to(device)
except Exception:
    # if model is a scripted module or already on correct device this may fail harmlessly
    pass
x = x.to(device)

with torch.no_grad():
    logit = model(x)            # (1,)
    prob = torch.sigmoid(logit).item()

print(f"Flood probability for {subdistrict_name}: {prob:.4f} on date {date.date()}")

Flood probability for กระทุ่มราย: 0.3051 on date 2024-01-01
